In [5]:
import sys
sys.path.append("..")
import json
import urllib.request
from IPython.display import clear_output
import sources
from countries.list import Countries
from data.Frame import Frame
from data.Row import Row
from data.Cell import Cell
from data.Stats import calculate_weighted_average as wa
c = Countries()
c.
from sources.worldbank.indicators import get_data_frame_wide as getdfw

In [6]:
def generate_weighted_average_table(indicator, weight, groups, start, end, export_to_excel=False):
    # We will store the final result here
    ndf = Frame()
    # Now we get the data that we want to average by region (defined in the groups)
    # and also total
    for g in groups:    
        # Data points
        df_data = getdfw(name=indicator, 
                    start=start, end=end, 
                    countries=c.get_groups(g),
                    label_field='entity',
                    value_field='value',
                    column_field='date'
                    )
        # Data to weight
        df_weight = getdfw(name = weight, 
                    countries = c.get_groups(g),
                    start=start, end=end,
                    label_field='entity',
                    value_field='value',
                    column_field='date'
                    )

        df_av = wa(df_data, df_weight, 'entity')

        # Change weigthed average from position
        r = df_av.rows[len(df_av.rows)-1]    
        for i in range(1, len(r.cells)-2,  3):
            r.cells[i] =  Cell(column=r.cells[i].column.name, value=r.cells[i+2].value.value)
        df_av.rows[len(df_av.rows)-1] = r

        for r in df_av.rows:
            ndf.rows.append(r)  

    cs = []        

    # Average for all groups

    for g in groups:
        for ctry in c.get_groups(g):
            cs.append(ctry)

    df_data = getdfw(name=indicator, 
                    start=start, end=end, 
                    countries=cs,
                    label_field='entity',
                    value_field='value',
                    column_field='date'
                   )

    df_weight = getdfw(name = weight, 
                    countries = cs,
                    start=start, end=end,
                    label_field='entity',
                    value_field='value',
                    column_field='date'
                    )

    df_av = wa(df_data, df_weight, 'entity')

    # Change weigthed average from position
    r = df_av.rows[len(df_av.rows)-1]   

    for i in range(1, len(r.cells)-2,  3):
        r.cells[i] = Cell(column=r.cells[i].column.name, value=r.cells[i+2].value.value)
        
    ndf.rows[len(ndf.rows)-1] = r
    
    # Produce dataframe that will be resturned as a result of the function
    final_df = Frame()    
    final_df.add_column(ndf.get_column('entity'))
    
    for i in range(end-start):        
        final_df.add_column(ndf.get_column('%s%s.0_value' % (str(start)[0:4-len(str(i))],i)))
        
    if export_to_excel == True:
        s = ""
        for g in groups:    
            for i in g:
                s =  i + "_" + s     
        final_df.name = "data/%s-%s-%s-%s-%s" % (indicator, weight, start, end, s)
        final_df.to_xlsx()
                            
    return final_df

In [7]:
start = 2000
end = 2018
indicator = 'SE.ADT.LITR.FE.ZS'
weight = 'SP.POP.TOTL'
groups=[['LDCs', 'Africa'],['LDCs', 'Asia and the Pacific'],['LDCs', 'America']]
groups_odc=[['Developing excluding LDCs']]
#groups=[['LDCs2017', 'Africa'],['LDCs2017', 'Asia and the Pacific'],['LDCs2017', 'America']]

In [8]:
generate_weighted_average_table(indicator, weight, groups, start, end, True)
generate_weighted_average_table(indicator, weight, groups_odc, start, end, True)